



*  **Not unidirectional, instead Bidirectional**
*  **Uses Masked LM and Next Sentence Prediction (NSP)** 
*  **It uses Encoder and Decoder. Encoder takes over priority since it is a pre - trained model** 
*  **Encoder collects words, and learns the nature. Decoder helps in the classification of words** 





In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google import colab
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/FYP

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/FYP


In [ ]:
import torch

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
else:
    print('Using CPU.')
    device = torch.device("cpu")

There are 1 GPU(s) available.


 **Makes use of Transformers library which build relationship between words**

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 33.7 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 596 kB 38.8 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd

tweet_train=pd.read_csv('cyberbullying_tweets.csv')

In [ ]:
tweets = tweet_train.TWEET_TEXT.values
labels = tweet_train.RESULT.values

 **Each word is converted to a unique token, and each token has a unique ID**

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tweetid = []
for tweet in tweets:
  encoded_tweet = tokenizer.encode(tweet,add_special_tokens = True,)
  tweetid.append(encoded_tweet)

print('Original: ', tweets[0])
print('Token IDs:', tweetid[0])

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors


Original:  In other words #katandandre, your food was crapilicious! #mkr
Token IDs: [101, 1999, 2060, 2616, 1001, 29354, 8943, 4859, 2890, 1010, 2115, 2833, 2001, 10231, 18622, 18436, 999, 1001, 12395, 2099, 102]


 **Padding is done to make all sentences of same length**

In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 64

print('\n Truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

tweetid = pad_sequences(tweetid, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")


 Truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0


 **Mask the encoded tokens for the model to understand the context**

In [ ]:
masks = []
for tweet in tweetid:
  mask = [int(token_id > 0) for token_id in tweet]
  masks.append(mask)

 **Split the dataset into training and testing datasets**

In [ ]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(tweetid, labels, random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(masks, labels, random_state=2018, test_size=0.1)

**Since we are using PyTorch, convert the data into tensors (data structure used to contain numerical data)**

In [ ]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2, 
    output_attentions = False, 
    output_hidden_states = False, 
)

model.cuda()
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

from transformers import get_linear_schedule_with_warmup

epochs = 4

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

**Batches allocation, and usage of the pre - trained BERT model**

In [ ]:
def accuracy(preds, labels):
    pred = np.argmax(preds, axis=1).flatten()
    labels = labels.flatten()
    return np.sum(pred == labels) / len(labels)

import random

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  total_loss = 0
  model.train()
  for step, batch in enumerate(train_dataloader):
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}. '.format(step, len(train_dataloader)))
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    model.zero_grad()        

    outputs = model(b_input_ids, 
                token_type_ids=None, 
                attention_mask=b_input_mask, 
                labels=b_labels)

    loss = outputs[0]
    total_loss += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    #print("  Average training loss: {0:.2f}".format(avg_train_loss))
    #print("validation")

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

  Batch    50  of  1,342. 
  Batch   100  of  1,342. 
  Batch   150  of  1,342. 
  Batch   200  of  1,342. 
  Batch   250  of  1,342. 
  Batch   300  of  1,342. 
  Batch   350  of  1,342. 
  Batch   400  of  1,342. 
  Batch   450  of  1,342. 
  Batch   500  of  1,342. 
  Batch   550  of  1,342. 
  Batch   600  of  1,342. 
  Batch   650  of  1,342. 
  Batch   700  of  1,342. 
  Batch   750  of  1,342. 
  Batch   800  of  1,342. 
  Batch   850  of  1,342. 
  Batch   900  of  1,342. 
  Batch   950  of  1,342. 
  Batch 1,000  of  1,342. 
  Batch 1,050  of  1,342. 
  Batch 1,100  of  1,342. 
  Batch 1,150  of  1,342. 
  Batch 1,200  of  1,342. 
  Batch 1,250  of  1,342. 
  Batch 1,300  of  1,342. 
  Batch    50  of  1,342. 
  Batch   100  of  1,342. 
  Batch   150  of  1,342. 
  Batch   200  of  1,342. 
  Batch   250  of  1,342. 
  Batch   300  of  1,342. 
  Batch   350  of  1,342. 
  Batch   400  of  1,342. 
  Batch   450  of  1,342. 
  Batch   500  of  1,342. 
  Batch   550  of  1,342. 
 